<center><h1>QBUS6850 - Machine Learning for Business</h1></center>

# Tutorial 4 - Linear Regression 2, Feature Extraction/Prepraration, Logistic Regression

## Ordinary Least Squares Drawbacks

$$min_{\boldsymbol  \beta} \frac{1}{2N}\| \mathbf{t-X} \boldsymbol \beta \|^2$$

$$\boldsymbol \beta =  (\mathbf X^T \mathbf X)^{-1} \mathbf X^T \mathbf t$$

OLS (Oridnary least squares) is an unbiased estimator by definition,
however this does not mean that it always produces a good model estimation.

OLS can completely fail or produce poor results under relatively
common conditions.

The shared point of failure among all these conditions is that
($\mathbf X^T \mathbf X$) is singular and OLS estimates depend on $(\mathbf X^T \mathbf X)^{-1}$. When $(\mathbf X^T \mathbf X)$ is
singular we cannot compute its inverse.

This happens frequently due to:
- collinearity i.e. predictors (features) are not independant
- $d > N$ i.e. number of features exceeds number of observations

Let $\mathbf M = \mathbf X^T \mathbf X \in \mathbb R^{d \times d}$ then $M_{ij} = \langle X_{(:, i)}, X_{(:, j)} \rangle$.

Or in other words each element of $\mathbf M$ encodes the similarity or distance from each feature vector to every other feature vector. When we have colinear features this can result in $rank(\mathbf M) < d$ meaning that $\mathbf M$ is singular and we cannot compute its inverse.

** Moreover OLS may be subject to outliers. This means that a single large outlier value can greatly affect our regression line. Therefore we know OLS is low (0) bias but high variance.  We might wish to
sacrifice some bias to achieve lower variance.**


## Ridge Regression

Ridge regression addresses both issues of high variance and invertibility.

By reducing the magnitude of the coefficients by using the $\ell_2$ norm we can reduce the coefficient variance and thus the variance of the estimator. Therefore the ridge regression objective function is

$$min_{\boldsymbol \beta} \frac{1}{2N}\| \mathbf{t - X} \boldsymbol \beta \|^2 + \frac{\lambda}{2N} \|\boldsymbol \beta \|^2$$

where $\lambda$ is a tunable parameter and controls the strength of the penalty. Therefore the solution is given by

$$\boldsymbol \beta =  (\mathbf X^T \mathbf X + \lambda \mathbf I)^{-1} \mathbf X^T \mathbf t$$

By reducing the variance of the estimator, ridge regression tends to be more "robust" against outliers than OLS since we have a solution with greater bias and less variance. Conventiantly this means that $\mathbf X^T \mathbf X$ is then modified to $\mathbf X^T \mathbf X + \lambda \mathbf I$. In other words we add a diagonal component to the matrix that we want to invert. This diagonal component is called the "ridge". This diagonal component increases the rank so that the matrix is no longer singular.

We can use cross validation to find the shrinkage parameter that we believe will generalise best to unseen data.

**About the Data**

The dataset is a collection of community and crime statistics from #http://archive.ics.uci.edu/ml/datasets/communities+and+crime

In [1]:
from sklearn.linear_model import LinearRegression, RidgeCV
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

#http://archive.ics.uci.edu/ml/datasets/communities+and+crime

crime = pd.read_csv('communities.csv', header=None, na_values=['?'])

Pre-processing steps

In [2]:
# Delete the first 5 columns
crime = crime.iloc[:, 5:]

# Remove rows with missing entries
crime.dropna(inplace=True)

Extract all the features available as our predictors and the response variable (number of violent crimes per capita)

In [3]:
# Get the features X and target/response y
X = crime.iloc[:, :-1]
t = crime.iloc[:, -1]

To evaluate model performance we will split the data into train and test sets.

Our procedure will follow:
- Fit model on training set
- Test performance via loss function values on test set

In [4]:
# Split data into train and test sets
X_train, X_test, t_train, t_test = train_test_split(X, t, random_state=1)

Fit OLS model and calculate loss values over test data

In [6]:
lm = LinearRegression()

lm.fit(X_train, t_train)

preds_ols = lm.predict(X_test)

print("OLS: {0}".format(mean_squared_error(t_test, preds_ols)/2))

OLS: 0.02733441765802363


Generate possible candidate sets for $\lambda$, fit the ridge model using cross validation on training set to find optimal $\lambda$ and calculate mse over test set.

In [7]:
alpha_range = 10.**np.arange(-2,3)

rregcv = RidgeCV(normalize=True, scoring='neg_mean_squared_error', alphas=alpha_range)

rregcv.fit(X_train, t_train)

preds_ridge = rregcv.predict(X_test)
print("RIDGE: {0}".format(mean_squared_error(t_test, preds_ridge)/2))

RIDGE: 0.013305662943587152


However RidgeCV can also be used without those parameters eg

In [8]:
rregcv = RidgeCV()

rregcv.fit(X_train, t_train)

preds_ridge = rregcv.predict(X_test)
print("RIDGE: {0}".format(mean_squared_error(t_test, preds_ridge)/2))

RIDGE: 0.012946592772763638


## Text Feature Extraction
- Bag of words
- Embedding

### Bag of Word
    Bag of words(1-gram) counts the words and keep the numbers and serve as the features.
    
    3 steps needs to be performed to get Bag of Word(BOW) features:
    1. Tokenizing: Segement the corpus into "words"
    2. Counting: Count the appearance frequecy of difference words.
    3. Mormalizing
    
    CountVectorizer from sklearn combine the tokenizing and counting.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [10]:
corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
    ]

X_txt = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names() == (
        ['and', 'document', 'first', 'is', 'one',
         'second', 'the', 'third', 'this'])

#X is the BOW feature of X
print(X_txt.toarray())

#the value is vocab id
print(vectorizer.vocabulary_)

#Unseen words are ignore
vectorizer.transform(['Something completely new.']).toarray()

[[0 1 1 1 0 0 1 0 1]
 [0 1 0 1 0 2 1 0 1]
 [1 0 0 0 1 0 1 1 0]
 [0 1 1 1 0 0 1 0 1]]
{'this': 8, 'is': 3, 'the': 6, 'first': 2, 'document': 1, 'second': 5, 'and': 0, 'third': 7, 'one': 4}


array([[0, 0, 0, 0, 0, 0, 0, 0, 0]])

Bag of Words features cannot caputer local information.
E.g. "believe or not" has the same features as "not or believe".
Bi-gram preserve more local information, which regrads 2 contagious words as one word in the vocabulary.
In the example, "believe or", "or not", "not or" and "or believe" are counted.
The feature is shown in the code below.

In [11]:
bigram_vectorizer = CountVectorizer(ngram_range=(2, 2),
                                        token_pattern=r'\b\w+\b', min_df=1)
analyze = bigram_vectorizer.build_analyzer()
print(analyze('believe or not a b c d e'))

['believe or', 'or not', 'not a', 'a b', 'b c', 'c d', 'd e']


Extract the features

In [12]:
X_txt_2 = bigram_vectorizer.fit_transform(corpus).toarray()
print(X_txt_2)

[[0 1 1 0 0 0 1 0 0 0 1 0]
 [0 0 1 0 1 1 0 1 0 0 1 0]
 [1 0 0 0 0 0 0 0 1 1 0 0]
 [0 1 0 1 0 0 1 0 0 0 0 1]]


### tf-idf
Some words has very high frequency(e.g. “the”, “a”, ”which”), therefore, carrying not much meaningful information about the actual contents of the document.

We need to compensate them to prevent the high-frequency shadowing other words.

$td-idf(t, d) = tf(t, d) \times idf(t)$

$idf(t) = log(\frac{1 + n_d}{1 + df(d, t)}) + 1$ (why +1 ?)
$n_d$ is the number of document.
$df(t)$ is the number of documents containing $t$.

Each row is normalized to have unit Euclidean norm:

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
counts = [[3, 0, 1],
            [2, 0, 0],
            [3, 0, 0],
            [4, 0, 0],
            [3, 2, 0],
            [3, 0, 2]
         ]
tfidf = transformer.fit_transform(counts)
print(tfidf)
print(tfidf.toarray())

  (0, 2)	0.5732079309279059
  (0, 0)	0.8194099510753754
  (1, 0)	1.0
  (2, 0)	1.0
  (3, 0)	1.0
  (4, 1)	0.8808994832762984
  (4, 0)	0.47330339145578754
  (5, 2)	0.8135516873095774
  (5, 0)	0.5814926070688599
[[0.81940995 0.         0.57320793]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [0.47330339 0.88089948 0.        ]
 [0.58149261 0.         0.81355169]]


A even more concise way to compute the tf-idf features. Combine counts and tf-idf

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
print(corpus)
X_txt_3 = vectorizer.fit_transform(corpus)
print(X_txt_3.toarray())

['This is the first document.', 'This is the second second document.', 'And the third one.', 'Is this the first document?']
[[0.         0.43877674 0.54197657 0.43877674 0.         0.
  0.35872874 0.         0.43877674]
 [0.         0.27230147 0.         0.27230147 0.         0.85322574
  0.22262429 0.         0.27230147]
 [0.55280532 0.         0.         0.         0.55280532 0.
  0.28847675 0.55280532 0.        ]
 [0.         0.43877674 0.54197657 0.43877674 0.         0.
  0.35872874 0.         0.43877674]]


## Feature Selection

- LASSO
- Elastic-Net
- Refer to lecture slides for their loss functions

We can take two approaches when using LASSO or Elastic-net for feature
selection. We can softly regularise the coefficients until a sufficient number
are set to 0. Fortunately using cross validation to optimise the regularisation
parameter lambda (called alpha in sklearn) usually results in many of the
features being ignored since their coefficient values are shrunk to 0.

Alternatively you can set a threshold value for the coefficient and find
a suitable regularisation parameter that meets this requirement.

We will take the path of cross validation.

Note that due to the shared L1/L2 regularisation of Elastic-Net it does not
aggressively prune features like Lasso. However in practice it often performs
better when used for regression prediction.

**NOTE: You can also use LASSO/Elastic-Net for regular regression tasks.**

Fit the LASSO model. Performed a train/test split and used CV on the train set to determine optimal parameters.

In [15]:
from sklearn.linear_model import LassoCV, ElasticNetCV

X = crime.iloc[:, :-1]
t = crime.iloc[:, -1]
X_train, X_test, t_train, t_test = train_test_split(X, t, random_state=1)

lascv = LassoCV()
lascv.fit(X_train, t_train)

preds_lassocv = lascv.predict(X_test)
print("LASSO: {0}".format(mean_squared_error(t_test, preds_lassocv)/2))
print("LASSO Lambda: {0}".format(lascv.alpha_))

/Users/jkan/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/jkan/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jkan/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jkan/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge

LASSO: 0.01298945881444861
LASSO Lambda: 0.0031688010361581157


/Users/jkan/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jkan/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jkan/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jkan/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not conver

Fit the Elastic-Net model

In [16]:
elascv = ElasticNetCV(max_iter=10000)
elascv.fit(X_train, t_train)

preds_elascv = elascv.predict(X_test)
print("Elastic Net: {0}".format(mean_squared_error(t_test, preds_elascv)/2))
print("Elastic Net Lambda: {0}".format(elascv.alpha_))

/Users/jkan/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Elastic Net: 0.012887092542898677
Elastic Net Lambda: 0.006337602072316219


Determine which columns were retained by each model

In [17]:
columns = X.columns.values

lasso_cols = columns[np.nonzero(lascv.coef_)]
print("LASSO Features: {0}".format(lasso_cols))

elas_cols = columns[np.nonzero(elascv.coef_)]
print("Elastic Features: {0}".format(elas_cols))

LASSO Features: [  7   8  20  22  33  37  39  43  49  55  73  76  95 107 113 121 124 125]
Elastic Features: [  7   8  20  22  33  37  39  43  46  48  49  50  55  73  76  82  95 107
 113 121 124 125]


## Regression vs Classification

- Logistic Regression

**WARNING: Do not use regression for classification tasks.**

In general it is ill advised to use linear regression for
classification tasks. Regression learns a continious output variable from
a predefined linear (or higher order) model. It learns the parameters
of this model to predict an output.

Classification on the other hand is not explicity interested in the
underlying generative process. Rather it is a higher abstraction. We are not
interested in the specific value of something. Instead we want to assign
each data vector to the most likely class.

Logistic regression provides us with two desirable properties:
- the output of the logistic function is the direct probability of the data vector belonging to the success case

$f(x, \beta) = \frac {1}{1+e^{-(\beta_0 + \beta_1 x)}}$

- the logistic function is non-linear and more flexible than a linear regression, which can improve classification accuracy and is often more robust to outliers. 

**About the dataset**

The data shows credit card loan status for many accounts with three features: student, balance remaining and income.

In [18]:
from sklearn.linear_model import LogisticRegression

df = pd.read_csv('Default.csv')

Convert the student category column to Boolean values

In [19]:
df.student = np.where(df.student == 'Yes', 1, 0)

Use the balance feature and set the default status as the target value to predict. You could also use all available features if you believe that they are informative.

In [20]:
X = df[['balance']]
t = df[['default']]

X_train, X_val, t_train, t_val = train_test_split(X, t, test_size=0.3, random_state=1)

Fit the Logistic Regression model

In [21]:
log_res = LogisticRegression()

log_res.fit(X_train, t_train)

/Users/jkan/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jkan/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

Predict probabilities of default.

predict_proba() returns the probabilities of an observation belonging to each class. This is computed from the logistic regression function (see above).

predict() is dependant on predict_proba(). predict() returns the class assignment based on the proability and the decision boundary. In other words predict returns the most likely class i.e. the class with greatest probability or probability > 50%.

In [22]:
prob = log_res.predict_proba(pd.DataFrame({'balance': [1200, 2500]}))
print(prob)
print("Probability of default with Balance of 1200: {0:.2f}%".format(prob[0,1] * 100))
print("Probability of default with Balance of 2500: {0:.2f}%".format(prob[1,1] * 100))

outcome = log_res.predict(pd.DataFrame({'balance': [1200, 2500]}))
print("Assigned class with Balance of 1200: {0}".format(outcome[0]))
print("Assigned class with Balance of 2500: {0}".format(outcome[1]))

[[0.96903859 0.03096141]
 [0.10491157 0.89508843]]
Probability of default with Balance of 1200: 3.10%
Probability of default with Balance of 2500: 89.51%
Assigned class with Balance of 1200: 0
Assigned class with Balance of 2500: 1


We can evaluate classification accuracy using confusion matrix (to be explanined in week 4 lecture) and the classification report

In [24]:
from sklearn.metrics import confusion_matrix

pred_log = log_res.predict(X_val)

print(confusion_matrix(t_val,pred_log))

[[2903    6]
 [  66   25]]


In [25]:
from sklearn.metrics import classification_report  

print(classification_report(t_val, pred_log, digits=3))

              precision    recall  f1-score   support

           0      0.978     0.998     0.988      2909
           1      0.806     0.275     0.410        91

   micro avg      0.976     0.976     0.976      3000
   macro avg      0.892     0.636     0.699      3000
weighted avg      0.973     0.976     0.970      3000

